# Large Model Example

Large models (schedules >~ 2 weeks) create a large number of variables due to the exponential variable scaling inherit to LMAS consumption constraints.

To solve large models a number of options exist:
- Solve with no LMAS constraints
    - Allows for model solutions, but likely will create impossible production schedules
- Solve smaller models sequentially for full production schedule
    - Break full schedule into ~2 week schedules and solve sequentially building off of previous solutions which are fixed in time
- Use sequential models to reduce model `horizon`
    - Long schedules require large horizon (maximum time provided for a model solution). Reducing the maximum horizon can greatly impact model solution time and can be informed by the sequential model approach

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %%
from scheduleopt import ScheduleModel
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import json
import numpy as np
import pandas as pd
from tqdm import tqdm


with open("input_lmas_ramp_updated") as f:
    inputs = json.load(f)
# inputs["forecast"] = [["M07A5", 60361, 744]]
# inputs["forecast"] = inputs["forecast1"]
# print(inputs["forecast"])
# with open("input_test/input_sample.json") as f:
#     inputs = json.load(f)

#model = ScheduleModel(inputs, time_scale_factor=4)

In [4]:
inputs["forecast"] =   [
        ["N82B1",  50000, 744]  
    ]

In [ ]:
#Do not RUN. Its just backup of variables 

["M07A5",  100000, 744],
        ["B05Y5",  100000, 744],
         ["C06F7",  10000, 372],
        ["W07N6",  10000, 372]
        
        ["M07A5",  205534, 744],
        ["M09C2", 45470, 744],
        ["A09B4", 96157, 744],
        ["J07B1",  27246, 744],
        ["K05B8",  27397, 744],
        ["F07G4",  43816, 744],
        ["D07D2",  39786, 744]
        
        
        
        ,
        ["K00Z3", 20000, 744],
        ["S80V4", 20000, 744],
        ["A03V8",  20000, 744], 
119,330.03 
76,974.01 
52,194.58 
51,575.17 


In [5]:
model = ScheduleModel(inputs, time_scale_factor=4)

In [6]:
# %%
# sol = model.solve_minimize_delivery_miss(max_time_in_seconds=None, verbose=True)
sol = model.solve_least_time_schedule(max_time_in_seconds=1500, number_of_search_workers=None, verbose=True, enforce_consumption_constraint=False)
# %%


Starting CP-SAT solver v9.6.2534
Parameters: random_seed: 1 max_time_in_seconds: 1500 log_search_progress: true num_search_workers: 16

Initial optimization model '': (model_fingerprint: 0xeb9f3c49e45a1334)
#Variables: 68 (#ints:1 in objective)
  - 9 Booleans in [0,1]
  - 44 in [0,324]
  - 15 constants in {8,20,26} 
#kInterval: 21 (#enforced: 21)
#kLinMax: 2 (#expressions: 3)
#kLinear1: 6 (#enforced: 6)
#kLinear2: 38 (#enforced: 36)
#kLinear3: 16 (#enforced: 15)
#kNoOverlap: 3 (#intervals: 15, #optional: 15)

Starting presolve at 0.00s
Unsat after presolving constraint #75 (warning, dump might be inconsistent): lin_max { target { vars: 66 coeffs: 1 } }

Presolve summary:
  - 0 affine relations were detected.
  - rule 'presolve: iteration' was applied 1 time.
Problem closed by presolve.

CpSolverResponse summary:
status: INFEASIBLE
objective: NA
best_bound: NA
integers: 0
booleans: 0
conflicts: 0
branches: 0
propagations: 0
integer_propagations: 0
restarts: 0
lp_iterations: 0
walltime:

In [7]:
mask = sol._job_schedule["MIN"] == "LMAS"
sol._job_schedule = sol._job_schedule.loc[~mask]
jobs_chart = sol.visualize_jobs()
machines_chart = sol.visualize_machines()
jobs_chart

AttributeError: 'ScheduleSolution' object has no attribute '_job_schedule'

In [7]:
machines_chart

alt.Chart(...)

## Previous Schedule

Use initial model with no LMAS constraints to provide starting point for serious of smaller models.

Smaller models will be built sequentially over time periods less than the full model time period to provide solvable constraints. 

Final model solution created using new maximum horizon from sequential model.

In [6]:
jobs = sol.job_schedule.loc[sol.job_schedule["MIN"] != "LMAS"].sort_values("Start")[["MIN", "JobId"]].drop_duplicates()
jobs.shape[0]

47

In [7]:
limits = [12, 24, 36, 48]
prev_n = 0
forecasts = []
for n in limits:
    jobs_data = jobs.iloc[prev_n:n]
    num_batches = jobs_data["MIN"].value_counts()
    batches = pd.Series(sol.input_data.batches)
    amounts = (num_batches * batches).dropna().astype(int).reset_index()
    amounts["time"] = 744
    forecasts.append(amounts.to_dict(orient="split")["data"])
    prev_n = n

In [ ]:
forecasts

[[['A09B4', 10653, 744],
  ['F07G4', 13832, 744],
  ['K05B8', 47620, 744],
  ['M07A5', 81498, 744],
  ['M09C2', 8499, 744]],
 [['D07D2', 23208, 744],
  ['F07G4', 41496, 744],
  ['M07A5', 81498, 744],
  ['M09C2', 8499, 744]],
 [['A09B4', 63918, 744], ['D07D2', 7736, 744], ['J07B1', 32475, 744]],
 [['A09B4', 31959, 744],
  ['D07D2', 15472, 744],
  ['M07A5', 54332, 744],
  ['M09C2', 33996, 744]]]

In [ ]:
solutions = []
max_times = [1500, 1500, 3000, 3000]
prev_sol = None
for max_time, forecast in tqdm(zip(max_times, forecasts)):
    inputs["forecast"] = forecast
    # %%
    if prev_sol is None:
        part_model = ScheduleModel(inputs, time_scale_factor=4)
    else:
        part_model = ScheduleModel(inputs, time_scale_factor=4, previous_schedule=prev_sol.job_schedule)
    # sol = model.solve_minimize_delivery_miss(max_time_in_seconds=None, verbose=True)
    solb = part_model.solve_least_time_schedule(max_time_in_seconds=max_time, number_of_search_workers=None, verbose=False, enforce_consumption_constraint=True)
    solutions.append(solb)
    prev_sol = solb
    jobs_chart = solb.visualize_jobs()
    machines_chart = solb.visualize_machines()
    jobs_chart.display()
    machines_chart.display()

0it [04:59, ?it/s]


AttributeError: 'ScheduleSolution' object has no attribute '_job_schedule'

In [ ]:
prev_sol.solver.StatusName()

In [ ]:
solutions[-2].job_schedule.to_csv("sol1_schedule.csv", index=False)

In [ ]:
prev_sol=solutions[0]
part_model = ScheduleModel(inputs, time_scale_factor=4, previous_schedule=prev_sol.job_schedule)
# sol = model.solve_minimize_delivery_miss(max_time_in_seconds=None, verbose=True)
solb = part_model.solve_least_time_schedule(max_time_in_seconds=2500, number_of_search_workers=None, verbose=True, enforce_consumption_constraint=True)

In [ ]:
len(solutions)

In [ ]:
solb = part_model.solve_least_time_schedule(max_time_in_seconds=1500, number_of_search_workers=None, verbose=False, enforce_consumption_constraint=True)

In [ ]:
solutions[-1].job_schedule.to_csv("final_schedule.csv"),index=False)